In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.distributions import Dirichlet, Bernoulli, Uniform
import pandas as pd
from tqdm import tqdm as tm

In [ ]:
from src import Simulation_dupe as sim
from src import Dir_Reg
from src import Align
from src import visualize_latent_space as vls

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("mps") if torch.backends.mps.is_available() else device

print(device)

The RGD stuff seems to have 2 stability-ish issues that needs to be addressed:
1. once in a while, it fails to minimize the out-of-simplex loss: This seems to be a result of the ReLU loss being too rigid, as the softplus loss with a large beta suffers a similar problem
2. it is not really affected by the initial guess: This is somewhat okay under low-dimensional settings, because we would simply get a permuted alignment instead --> XW * (W^T)B, we get (W^T)B
3. sometimes the result is NA: this is because we are using the true beta to initialize, but sometimes the recovered alignment is XW which corresponds to (W^T)B instead of B

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.distributions import Dirichlet, Bernoulli, Uniform
import pandas as pd
from tqdm import tqdm as tm
from src import Simulation_dupe as sim
from src import Dir_Reg
from src import Align
from src import visualize_latent_space as vls

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("mps") if torch.backends.mps.is_available() else device

print(device)

N = 1

temp = sim.ABC_Monte_Carlo.consistency_T2(number_of_iterations = N, 
                                          nodes_set = [300],
                                          beta = [1,1,-4, 5],
                                          alpha_0 = [[10, 1, 1], [1, 10, 1], [1, 1, 10]],
                                          seeded = True,
                                          constrained = False,
                                          oracle_lat_pos = True,
                                          oracle_align = True,
                                          no_oracle = True)

cuda


300: 100%|██████████| 1/1 [00:00<00:00,  1.04it/s]


In [10]:
temp.MC_result.est["info_lost"][:21]
# temp.MC_result.est["info_lost"][21: 42]
# temp.MC_result.est["info_lost"][42: 63]


0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
Name: info_lost, dtype: float32

In [11]:
model = sim.ABC(time = 2,
            nodes = 3,
            beta = [1,1,-4, 5],
            alpha_0 = [[10, 1, 1], [1, 10, 1], [1, 1, 10]])
C = model.settings.C

res = torch.tensor(temp.MC_result.est["B_est"]).reshape(N*3, 21).T

torch.linalg.solve(C.T @ C, C.T @ res).T

tensor([[ 0.9761,  1.0277, -3.9683,  5.3105],
        [ 1.1315,  1.1236, -3.2146,  1.1226],
        [ 1.1124,  1.0130, -3.4994,  2.7868]])

In [ ]:
i = 1
while i <= 9:
    model = sim.ABC(time = 2,
                nodes = 3,
                beta = [1,1,-4, 5],
                alpha_0 = [[10, 1, 1], [1, 10, 1], [1, 1, 10]])
    p = model.settings.p
    torch.manual_seed(i)
    model.update_settings(nodes = 300)
    Z0 = model.synth_data["lat_pos"][0,]
    Z1 = model.synth_data["lat_pos"][1,]
    Y0 = model.synth_data["obs_adj"][0,]
    Y1 = model.synth_data["obs_adj"][1,]


    Z_0 = Align.No_Oracle(Y0, (p-1)).aligned
    X_0 = sim.ABC.gen_X(Y0, Z0, 3)
    Z_1 = Align.No_Oracle(Y1, (p-1)).aligned

    # vls.lat_vis(Z0, 3, -1, "Reds")
    # vls.lat_vis(Z1, 3, -1, "Blues")

    test = Dir_Reg.fit(X_0, Z_1)

    res = test.est_result["estimate"].reshape(-1)
    print(torch.linalg.solve(C.T @ C, C.T @ res).T)
    i += 1

Another issue of the no_oracle monte-carlo simulation remains. For some unknown reason, we get the following error:

    --> 270 est_NO = est_no_oracle.est_result["estimate"].reshape(-1)

    271 fish_NO = est_no_oracle.est_result["fisher_info"].reshape(1, -1)
    
    272 method_NO = torch.tensor([[0,0,1]])

IndexError: too many indices for tensor of dimension 2

corresponding seed: 977

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.distributions import Dirichlet, Bernoulli, Uniform
import pandas as pd
from tqdm import tqdm as tm

from src import Simulation as sim
from src import Dir_Reg
from src import Align
from src import visualize_latent_space as vls

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("mps") if torch.backends.mps.is_available() else device

print(device)

In [ ]:
model = sim.ABC(time = 2,
                nodes = 3,
                beta = [1,1,-4, 5],
                alpha_0 = [[10, 1, 1], [1, 10, 1], [1, 1, 10]])
p = model.settings.p
torch.manual_seed(9)
model.update_settings(nodes = 300)
Z0 = model.synth_data["lat_pos"][0,]
Z1 = model.synth_data["lat_pos"][1,]
Y0 = model.synth_data["obs_adj"][0,]
Y1 = model.synth_data["obs_adj"][1,]


Z_0 = Align.No_Oracle(Y0, (p-1)).aligned
X_0 = sim.ABC.gen_X(Y0, Z0, 3)
Z_1 = Align.No_Oracle(Y1, (p-1)).aligned

vls.lat_vis(Z0, 3, -1, "Reds")
vls.lat_vis(Z1, 3, -1, "Blues")

test = Dir_Reg.fit(X_0, Z_1)

model = sim.ABC(time = 2,
            nodes = 3,
            beta = [1,1,-4, 5],
            alpha_0 = [[10, 1, 1], [1, 10, 1], [1, 1, 10]])
C = model.settings.C
res = test.est_result["estimate"].reshape(-1)
print(torch.linalg.solve(C.T @ C, C.T @ res).T)

In [ ]:
test.est_result